# Indice m²
### This notebook create a buffer for each EP type, acording to its radius of influence.

In [1]:
# import libraries
import pandas as pd
import geopandas as gpd

## Load file

In [2]:
df = gpd.read_file("./CapasEditadas/EspaciosPublicos_v11.geojson")
df.head(3)

,full_id,fuente,distrito,evp,tipo,nombre,limite,area_m2,sis_bool,sis_nam,sis_area_m2,barpop_cant,barpop_nam,bus50_cant,bus500_cant,observaciones,geometry
0,r13232486,OSM,Lomas de Zamora,1.0,Plaza,Plaza Carlos Pellegrini,1,6932.85,0,None,NaN,0.0,None,0.0,7.0,None,"MULTIPOLYGON (((-58.42685 -34.79316, -58.42684..."
1,w28051684,OSM,Lomas de Zamora,1.0,Plaza,"Plaza Libertad Y Democracia, Dr. Raúl Alfonsín",1,14263.38,0,None,NaN,0.0,None,0.0,2.0,None,"MULTIPOLYGON (((-58.41578 -34.75867, -58.41435..."
2,w28051696,OSM,Lomas de Zamora,1.0,Plaza,Plaza de los Italianos,1,7883.71,0,None,NaN,0.0,None,0.0,9.0,None,"MULTIPOLYGON (((-58.42691 -34.76192, -58.42614..."


In [3]:
# importar capas de municipios y límite antrópico
gdf_muni = gpd.read_file("../1_Datos/ACUMAR/LIMITE_POLITICO.shp")
# filtro de los municipios de interés 
gdf_muni = gdf_muni.loc[(gdf_muni.DISTRITO == "LOMAS DE ZAMORA") | (gdf_muni.DISTRITO == "LANUS")]

In [4]:
df_radios_lanus = gpd.read_file("../1_Datos/RadiosCensales_Antro/lanus_cuenca_antropica.shp")
df_radios_lanus.rename(columns={'Proyecci¢':'Proyeccion'}, inplace=True)
df_radios_lomas = gpd.read_file("../1_Datos/RadiosCensales_Antro/lomas_cuenca_antropica.shp")
df_radios = pd.concat([df_radios_lanus, df_radios_lomas])
print(df_radios.crs)
df_radios = df_radios.to_crs("EPSG:5347")
df_radios['area_radio'] = df_radios.area
df_radios = df_radios.to_crs("EPSG:4326")
print(len(df_radios))
df_radios.head()

epsg:22183
763


,toponimo_i,link,varon,mujer,totalpobl,hogares,viviendasp,viv_part_h,ID,Pob_2010,Proyeccion,Superficie,geometry,area_radio
0,302170,064343303,467.0,518.0,985.0,345.0,379.0,331.0,64340301,985,992,98272.089,"POLYGON ((-58.40740 -34.70832, -58.40764 -34.7...",98323.492127
1,305820,064340211,462.0,530.0,992.0,359.0,429.0,347.0,64340211,992,999,110194.527,"POLYGON ((-58.40410 -34.67499, -58.40326 -34.6...",110252.491165
2,305857,064340212,396.0,438.0,834.0,299.0,370.0,297.0,64340212,834,840,93941.812,"POLYGON ((-58.40098 -34.67732, -58.40189 -34.6...",93991.367470
3,305943,064340210,250.0,293.0,543.0,211.0,248.0,194.0,64340210,543,547,68710.439,"POLYGON ((-58.40572 -34.67379, -58.40647 -34.6...",68746.494489
4,305947,064340101,942.0,924.0,1866.0,531.0,418.0,397.0,64340101,1866,1880,495566.120,"POLYGON ((-58.41692 -34.66108, -58.41688 -34.6...",495825.460221


# Functions

In [5]:
def calcularDataArea (df, outpath):
    df_ep_areas = df
    # overlay radios polygons with ep polygons
    df_radios_ep = gpd.overlay(df_radios, df_ep_areas, how='intersection')
    # calculate areas
    df_radios_ep = df_radios_ep.to_crs("EPSG:5347")
    df_radios_ep['area_radep'] = df_radios_ep.area
    df_radios_ep = df_radios_ep.loc[df_radios_ep['area_radep'] != 0].reset_index(drop=True)
    df_radios_ep = df_radios_ep.to_crs("EPSG:4326")
    # calculate area % and m² ep % over the radio
    df_radios_ep['area_radporc'] = df_radios_ep.area_radep / df_radios_ep.area_radio  
    df_radios_ep['radep_area_epm2'] = (df_radios_ep.area_radep * df_radios_ep.area_m2) # sup pol * m² ep

    # calculate sum of values and merge with radios geometries
    df_radios_data = df_radios_ep.groupby(['link']).agg({'area_radep':['sum'], 'radep_area_epm2':['sum'], 'area_radporc':['sum']})
    df_radios_data = df_radios[['link', 'Proyeccion', 'area_radio', 'geometry']].merge(df_radios_data, how='left', on='link')
    df_radios_data.rename(columns={'Proyeccion':'rad_pob', 'area_radio':'rad_area',
                                df_radios_data.columns[4]:'radep_area', df_radios_data.columns[5]:'radep_area_epm2', df_radios_data.columns[6]:'radep_areaporc'}, inplace=True)
    # calculate m² ep x hab
    df_radios_data['radep_pob'] = round(df_radios_data['radep_areaporc'] * df_radios_data['rad_pob'])
    df_radios_data['ep_areapond_radep'] = df_radios_data['radep_area_epm2'] / df_radios_data['radep_area']
    df_radios_data['ep_areapond_rad'] = df_radios_data['radep_area_epm2'] / df_radios_data['rad_area']
    df_radios_data['m2_hab_radep'] = df_radios_data['ep_areapond_radep'] / df_radios_data['radep_pob']
    df_radios_data['m2_hab'] = df_radios_data['ep_areapond_rad'] / df_radios_data['rad_pob']
    # clean nulls
    df_radios_data.radep_pob.fillna(0, inplace=True)
    df_radios_data.radep_area.fillna(0, inplace=True)
    df_radios_data.radep_areaporc.fillna(0, inplace=True)
    df_radios_data.radep_area_epm2.fillna(0, inplace=True)
    df_radios_data.ep_areapond_radep.fillna(0, inplace=True)
    df_radios_data.ep_areapond_rad.fillna(0, inplace=True)
    df_radios_data.m2_hab_radep.fillna(0, inplace=True)
    df_radios_data.m2_hab.fillna(0, inplace=True)
    # filter columns
    df_radios_data = df_radios_data[['link', 'rad_pob', 'rad_area', 'radep_pob', 'radep_area', 'radep_areaporc', 'radep_area_epm2', 'ep_areapond_radep', 'ep_areapond_rad', 'm2_hab_radep', 'm2_hab', 'geometry']]
    df_radios_data.to_file(outpath, driver='GeoJSON')
    df_radios_data

In [6]:
def calcularDataTiempo(path_time, path_m2, outpath):
    # import time data
    df_time = gpd.read_file(path_time)

    df_radios_time = gpd.overlay(df_radios, df_time, how='intersection')
    df_radios_time = df_radios_time.to_crs("EPSG:5347")
    df_radios_time['area_radtime'] = df_radios_time.area
    df_radios_time = df_radios_time.loc[df_radios_time['area_radtime'] != 0].reset_index(drop=True)
    df_radios_time = df_radios_time.to_crs("EPSG:4326")
    df_radios_time['area_radporc'] = df_radios_time.area_radtime / df_radios_time.area_radio  
    df_radios_time['radep_area_time'] = (df_radios_time.area_radtime * df_radios_time.ToBreak) # sup pol * time

    df_time_data = df_radios_time.groupby(['link']).agg({'area_radtime':['sum'], 'radep_area_time':['sum'], 'area_radporc':['sum']})
    df_time_data = df_radios[['link', 'Proyeccion', 'area_radio', 'geometry']].merge(df_time_data, how='left', on='link')
    df_time_data.rename(columns={'Proyeccion':'rad_pob', 'area_radio':'rad_area',
                                df_time_data.columns[4]:'radtime_area', df_time_data.columns[5]:'time_area_min', df_time_data.columns[6]:'radtime_areaporc'}, inplace=True)
    df_time_data['radtime_pob'] = round(df_time_data['radtime_areaporc'] * df_time_data['rad_pob'])
    df_time_data['time_pond'] = df_time_data['time_area_min'] / df_time_data['rad_area'] + (20 * (1-df_time_data['radtime_areaporc']))
    df_time_data.radtime_area.fillna(0, inplace=True)
    df_time_data.time_area_min.fillna(0, inplace=True)
    df_time_data.radtime_areaporc.fillna(0, inplace=True)
    df_time_data.radtime_pob.fillna(0, inplace=True)
    df_time_data.time_pond.fillna(0, inplace=True)
    df_time_data = df_time_data[['link', 'rad_pob', 'rad_area', 'radtime_pob', 'radtime_area', 'radtime_areaporc', 'time_area_min', 'time_pond', 'geometry']]

    # load surface/hab data 
    path_load = path_m2
    df_radios_data = gpd.read_file(path_load)
    # merge surface/hab data with time data
    df_data = df_radios_data.merge(df_time_data[['link', 'radtime_pob', 'radtime_area', 'radtime_areaporc', 'time_area_min', 'time_pond']], on='link', how="left")
    df_data_2 = df_data[['link', 'rad_pob', 'rad_area', 'radep_pob', 'radep_area',
       'radep_areaporc', 'm2_hab_radep', 'm2_hab', 'geometry', 'radtime_pob',
       'radtime_area', 'radtime_areaporc', 'time_pond']]

    # create a time range field
    df_data_2['time_range'] = '15 - +'
    df_data_2.loc[df_data_2.time_pond <= 15, 'time_range'] = '10 - 15'
    df_data_2.loc[df_data_2.time_pond <= 10, 'time_range'] = '5 - 10'
    df_data_2.loc[df_data_2.time_pond <= 5, 'time_range'] = '0 - 5'
    # create a categorical field with the OSM stardart
    df_data_2['OMS_values'] = 'Malo'
    df_data_2.loc[df_data_2.m2_hab > 10, 'OMS_values'] = 'Bueno'
    df_data_2.loc[df_data_2.m2_hab > 15, 'OMS_values'] = 'Óptimo'

    # export data
    df_data_2 = df_data_2[['link', 'rad_pob', 'rad_area', 'radep_pob', 'radep_area',
        'radep_areaporc', 'm2_hab_radep', 'm2_hab', 'OMS_values', 'radtime_pob',
        'radtime_area', 'radtime_areaporc', 'time_pond', 'time_range',  'geometry']]
    df_data_2.to_file(outpath, driver='GeoJSON')

In [7]:
def create_clipped_buffers (df_evp_buffers, field_id, field_name, field_evp_area, geojson_output_path):
    
    if len(df_evp_buffers) <= 1 :
        df_union = df_evp_buffers[['geometry']]
    if len(df_evp_buffers) >1 :
        # create list of ids
        lista_id = list(df_evp_buffers[field_id])
        # overlay the two first polygons
        df_1 = df_evp_buffers.loc[df_evp_buffers[field_id] == lista_id[0], ['geometry']]
        df_2 = df_evp_buffers.loc[df_evp_buffers[field_id] == lista_id[1], ['geometry']]
        df_union = gpd.overlay(df_1, df_2, how='union').explode().reset_index(drop=True)
        print(len(df_union))
        # loop throw the rest of the polygons
        for pol in range(2,len(lista_id)):
            df_2 = df_evp_buffers.loc[df_evp_buffers[field_id] == lista_id[pol], ['geometry']]
            df_union = gpd.overlay(df_union, df_2, how='union').explode().reset_index(drop=True)

    # create a new id field
    df_union.reset_index(inplace=True)
    df_union.rename(columns={df_union.columns[0]:'id_pol'}, inplace=True)
    df_union.id_pol = df_union.apply(lambda row: "id_"+ str(row.id_pol), axis=1)
    # sjoin with buffers to sum area
    df_evp_buffers_minus1m = df_evp_buffers.to_crs("EPSG:5347")
    df_evp_buffers_minus1m.geometry = df_evp_buffers_minus1m.buffer(-1)
    df_evp_buffers_minus1m = df_evp_buffers_minus1m.to_crs("EPSG:4326")
    df_sj = gpd.sjoin(df_union, df_evp_buffers_minus1m[[field_name, field_evp_area, 'geometry']])

    # calculate name list and area sum
    data_nombres = df_sj.groupby(['id_pol'])[field_name].apply(lambda x: str(list(set(','.join(x).split(','))))).reset_index()
    data_area = df_sj.groupby(['id_pol'])[field_evp_area].sum().reset_index()
    # merge data with original poligons
    df_union_merged = pd.merge(df_union, data_nombres, on='id_pol', how='left')
    df_union_merged = pd.merge(df_union_merged, data_area, on='id_pol', how='left')
    df_union_merged.to_file(geojson_output_path, driver='GeoJSON')
    
    return df_union_merged

## Buffers according to radius

In [36]:
df['radio_m'] = 0

# EP radius
df.loc[df.tipo == 'Plazoleta', 'radio_m'] = 400
df.loc[df.tipo == 'Plaza', 'radio_m'] = 600
df.loc[df.tipo == 'Parque', 'radio_m'] = 800
df.loc[df.tipo == 'Parque con equipamiento', 'radio_m'] = 2000

df_ind = df.loc[df.radio_m != 0]
df_ind.reset_index(inplace=True, drop=True)
df_ind.tipo.unique()

df_ind = df_ind.to_crs("EPSG:5347")
df_ind.geometry = df_ind.buffer(df_ind['radio_m'])
df_ind = df_ind.to_crs("EPSG:4326")
df_ind.to_file("./Indice_m2/EP_buffers_all.geojson", driver='GeoJSON')

## Creating polygons with different m²

#### cruzar capa anterior y radios censales: interseccion
'''
radio sup total = 40% (area y pob) - 600m²
    pol 1 = sup 10% (area y pob) - EPm² = 300m²
    pol 2 = sup 10% (area y pob) - EPm² = 100m²
    pol 3 = sup 20% (area y pob) - EPm² = 200m²

    total = (10%/40%) * 300m² + (10%/40%) * 100m² + (20%/40%) * 200m² = 200
    total = [10% * 300m² + 10% * 100m² + 20% * 200m²] /40%= 200
'''

In [8]:
df_ind = gpd.read_file("./Indice_m2/EP_buffers_all.geojson")
create_clipped_buffers (df_ind, 'full_id', 'nombre', 'area_m2', "./Indice_m2/EP_buffers_all_union_sj_data_corr1m.geojson")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  # This is added back by InteractiveShellApp.init_path()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_new_instance()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silen

2


c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_new_instance()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_new_instance()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_n

,id_pol,geometry,nombre,area_m2
0,id_0,"POLYGON ((-58.46974 -34.73808, -58.46880 -34.7...",['Parque Soldado Brito / Parque de Santa Catal...,3243503.74
1,id_1,"POLYGON ((-58.46880 -34.73766, -58.46974 -34.7...",['Parque de Villa Albertina / Parque Albertina...,110908.00
2,id_2,"POLYGON ((-58.47076 -34.73861, -58.46974 -34.7...",['Parque Soldado Brito / Parque de Santa Catal...,3242123.55
3,id_3,"POLYGON ((-58.47149 -34.73718, -58.47148 -34.7...",['Parque de Villa Albertina / Parque Albertina...,109527.81
4,id_4,"POLYGON ((-58.47444 -34.74873, -58.47446 -34.7...","['Los 2 Hermanos', 'Parque Soldado Brito / Par...",3239025.72
...,...,...,...,...
2794,id_2794,"POLYGON ((-58.43941 -34.68115, -58.43941 -34.6...","['Plaza Giardino', 'Plaza del Futuro']",19000.48
2795,id_2795,"POLYGON ((-58.42211 -34.66388, -58.42212 -34.6...",['Plaza Ernesto Sábato'],367.28
2796,id_2796,"POLYGON ((-58.40493 -34.78033, -58.40478 -34.7...","['Parque Municipal Finky', '-']",55556.85
2797,id_2797,"POLYGON ((-58.40516 -34.78004, -58.40516 -34.7...","['Parque Municipal de Lomas de Zamora', '-']",244641.72


In [9]:
df_ep_areas = gpd.read_file("./Indice_m2/EP_buffers_all_union_sj_data_corr1m.geojson")
path_save = "./Indice_m2/Radios_EP_m2_v2.geojson"
calcularDataArea (df_ep_areas, path_save)

calcularDataTiempo("../3_RedGeometrica/evp_acce/geonet_poltime_distritos.shp", path_save, "./Indice_m2/Radios_EP_m2time_v2.geojson")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:4: UserWarning: `keep_geom_type=True` in overlay resulted in 7 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  after removing the cwd from sys.path.
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

# m2 por tipología

In [10]:
df_ind_all = gpd.read_file("./Indice_m2/EP_buffers_all.geojson")

for eptipo in list(df_ind_all.tipo.unique()):
    df_ind = df_ind_all.loc[df_ind_all.tipo == eptipo]
    path_save_buf = "./Indice_m2/EP_buffers_all_union_sj_data_"+eptipo+"_corr1m.geojson"
    create_clipped_buffers (df_ind, 'full_id', 'nombre', 'area_m2', path_save_buf)

    df_ep_areas = gpd.read_file(path_save_buf)
    path_save_m2 = "./Indice_m2/Radios_EP_m2_"+eptipo+"_v2.geojson"
    calcularDataArea (df_ep_areas, path_save_m2)

    path_load_m2time = "../3_RedGeometrica/evp_acce/geonet_poltime_distritos_antropico_"+eptipo+".shp"
    path_save_m2time = "./Indice_m2/Radios_EP_m2time_"+eptipo+"_v2.geojson"
    calcularDataTiempo(path_load_m2time, path_save_m2, path_save_m2time)

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  # This is added back by InteractiveShellApp.init_path()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_new_instance()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silen

2


c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_new_instance()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_new_instance()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_n

2


c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_new_instance()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_new_instance()
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  app.launch_n

2


c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\indexing.py:1765: 

# Parques Potenciales

In [6]:
# importar parques potenciales
df_parques_potenciales = gpd.read_file("./ep_parques_potenciales/EspaciosPublicos_PARQUE CON POT.shp")
# create buffers
df_parques_potenciales = df_parques_potenciales.to_crs("EPSG:5347")
df_parques_potenciales.geometry = df_parques_potenciales.buffer(800)
df_parques_potenciales = df_parques_potenciales.to_crs("EPSG:4326")
df_parques_potenciales.to_file("./Indice_m2/EP_PARQUE_CON_POTbuffers.geojson", driver='GeoJSON')

# calculate evp area /pob by radius
path_save = "./Indice_m2/EP_PARQUE_CON_POTbuffers_m2.geojson"
calcularDataArea (df_parques_potenciales, path_save)
# calculate time travel by radius
path_time = "../3_RedGeometrica/parques_potenciales/geonet_poltime_ParquePotenciales_disseolve.shp"
outpath = "./Indice_m2/EP_PARQUE_CON_POTbuffers_m2time.geojson"
calcularDataTiempo(path_time, path_save, outpath)

# Sistemas de EVP

In [8]:
# load evp buffers and dissolve by sist
df_eps_buffers = gpd.read_file("./Indice_m2/EP_buffers_all.geojson", driver='GeoJSON')
df_eps_buffers_sist = df_eps_buffers.loc[df_eps_buffers.sis_bool == 1].dissolve(
     by="sis_nam",
     aggfunc={
         "area_m2": ["max"]
     },
 ).reset_index()
df_eps_buffers_sist.reset_index(inplace=True)
df_eps_buffers_sist.rename(colu mns={df_eps_buffers_sist.columns[0]: 'id', df_eps_buffers_sist.columns[3]: 'area_m2'}, inplace=True)
df_eps_buffers_sist.to_file("./Indice_m2/EP_buffers_sist.geojson", driver='GeoJSON')

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [55]:
# cross buffers and calculate epv area
df_union_merged = create_clipped_buffers (df_eps_buffers_sist, 'id', 'sis_nam', 'area_m2', "./Indice_m2/EP_buffers_sist_union_sj_data.geojson")
df_union_merged.head()

# calculate evp area /pob by radius
path_save = "./Indice_m2/EP_SISTEMAS_buffers_m2.geojson"
calcularDataArea (df_union_merged, path_save)
# calculate time travel by radius
path_time = "../3_RedGeometrica/evp_sistemas/geonet_sistemas_ttravel_disseolve.shp"
outpath = "./Indice_m2/EP_SISTEMAS_buffers_m2time.geojson"
calcularDataTiempo(path_time, path_save, outpath)

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\pandas\core\indexing.py:1765: 